# Processing data for manual time lapse inversion
> Framework
- import baseline & survey
- process reciprocal error
- save into .protocol file
- gather baseline + survey for timelapse inversion


>> check difference inversion to define if we need baseline modeling

In [9]:
# Import functions

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.dates as mdates
from os.path import isfile, join, exists
from resipy import Project #to install: pip install resipy

# New function

def ER_filtering(data, DevTh, Pot, Curr, Stk, AppR):
    # Calculate resistance
    data['R'] = data['VMN (mV)'] / data['IAB (mA)']

    # Filtering in function of deviation
    idx = data['Dev. Rho (%)'] > DevTh
    data['R'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from stacking dev error threshold')

    # Filtering in function of potential
    idx = data['VMN (mV)'] == Pot
    data['R'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from negative VMN')

    # Filtering in function of current
    idx = data['IAB (mA)'] <= Curr
    data['R'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from negative IAB')

    # Filtering in function of stacking
    idx = data['IAB (mA)'] <= Stk
    data['R'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from stacking')

    # Filtering in function of app. resistivity
    idx = data['IAB (mA)'] >= AppR
    data['R'][idx] = np.nan
    print(f'Filtered out {idx.sum()} from too high resistivity')
    
    data = data.dropna(subset=['R'])
    # data = data[~np.isnan(data).any(axis=1)]

    return data

def saveSyscal(Stdata,Rcdata,sfile):
    d1 = np.column_stack(
        (Stdata['El-array'],
         Stdata['N° electrode A'],
         Stdata['N° electrode B'], 
         Stdata['N° electrode M'],
         Stdata['N° electrode N'],
         Stdata['Rho (Ohm.m)'],
         Stdata['Dev. Rho (%)'],
         Stdata['SP (mV)'],
         Stdata['VMN (mV)'],
         Stdata['IAB (mA)']))
    d2 = np.column_stack(
        (Rcdata['El-array'],
         Rcdata['N° electrode A'],
         Rcdata['N° electrode B'], 
         Rcdata['N° electrode M'],
         Rcdata['N° electrode N'], 
         Rcdata['Rho (Ohm.m)'],
         Rcdata['Dev. Rho (%)'],
         Rcdata['SP (mV)'],
         Rcdata['VMN (mV)'],
         Rcdata['IAB (mA)']))
    
    data = np.concatenate((d1,d2), axis=0)
    new_col = np.full((data.shape[0], 1), " ")
    data = np.hstack((new_col, data))

    with open(sfile, 'wt') as f:  
        f.write(",El-array, Spa.1, Spa.2, Spa.3, Spa.4, Rho, Dev., Sp, Vp, In\n")
        for row in data:
            f.write(','.join(map(str, row)) + '\n')

def saveProtocol(Stdata,Rcdata,sfile):
    nd1 = len(Stdata)
    d1 = np.column_stack(
        (np.arange(1, nd1 + 1),
         Stdata['xA (m)']+1,
         Stdata['xB (m)']+1, 
         Stdata['xM (m)']+1,
         Stdata['xN (m)']+1,
         Stdata['R']))#,
        #  Stdata['Dev. Rho (%)']))
    nd2 = len(Rcdata)+nd1
    d2 = np.column_stack(
        (np.arange(nd1+1, nd2 + 1),
         Rcdata['xA (m)']+1,
         Rcdata['xB (m)']+1, 
         Rcdata['xM (m)']+1,
         Rcdata['xN (m)']+1, 
         Rcdata['R']))#,
        #  Rcdata['Dev. Rho (%)']))
    
    data = np.concatenate((d1,d2), axis=0)
    
    nbD = len(data)
    with open(sfile, 'wt') as f:  
        f.write(f"{nbD}\n")
        for row in data:
            f.write('\t'.join(map(str, row)) + '\n')

> Data import

> Data filtering
- Deviation threshold: DevTh (in %)
- Positive VMN potential: Pot (in mV)
- Postive IAB current: Curr(in mA)
- More than X stack: Stk (nb)
- With App. Resistivity lower than Y: AppR (Ohm.m)

> Save

In [10]:
# Import data

date = ['2406', '2407', '2408']
Line = 'C'

Dtfolder = 'C:/Users/Marc/OneDrive - Colorado School of Mines/Documents/4-Colorado/2-GordonGulch/0-Data/24_ERdata/2-Txtfiles/'
Svfolder = 'C:/Users/Marc/OneDrive - Colorado School of Mines/Documents/4-Colorado/2-GordonGulch/0-Data/24_ERdata/24-Line'

# Data processing

DevTh = 10
Pot = 0
Curr = 0
Stk = 1
AppR = 999999 #use 999999 to not include this.

for d in date:
    print(d)
    # standard data
    file = os.path.join(Dtfolder + d +'-ER' + Line + '_stand.txt')
    Dst = pd.read_csv(file, delimiter=' ', encoding='latin-1')
    # reciprocal data
    file = os.path.join(Dtfolder + d +'-ER' + Line + '_recip.txt')
    Drp = pd.read_csv(file, delimiter=' ', encoding='latin-1')

    Dst = ER_filtering(Dst, DevTh, Pot, Curr, Stk, AppR)
    Drp = ER_filtering(Drp, DevTh, Pot, Curr, Stk, AppR)

    # Save file
    sfile = os.path.join(Svfolder + Line + '/Data/' + d + '-ER.dat')
    saveProtocol(Dst,Drp,sfile)


2406
Filtered out 66 from stacking dev error threshold
Filtered out 0 from negative VMN
Filtered out 0 from negative IAB
Filtered out 64 from stacking
Filtered out 0 from too high resistivity
Filtered out 3 from stacking dev error threshold
Filtered out 0 from negative VMN
Filtered out 0 from negative IAB
Filtered out 15 from stacking
Filtered out 0 from too high resistivity
2407
Filtered out 171 from stacking dev error threshold
Filtered out 614 from negative VMN
Filtered out 613 from negative IAB
Filtered out 631 from stacking
Filtered out 0 from too high resistivity
Filtered out 18 from stacking dev error threshold
Filtered out 0 from negative VMN
Filtered out 0 from negative IAB
Filtered out 0 from stacking
Filtered out 0 from too high resistivity
2408
Filtered out 513 from stacking dev error threshold
Filtered out 130 from negative VMN
Filtered out 129 from negative IAB
Filtered out 149 from stacking
Filtered out 0 from too high resistivity
Filtered out 43 from stacking dev error 

C:\Users\Marc\AppData\Local\Temp\ipykernel_19724\2511944939.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['R'][idx] = np.nan
C:\Users\Marc\AppData\Local\Temp\ipykernel_19724\2511944939.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['R'][idx] = np.nan
C:\Users\Marc\AppData\Local\Temp\ipykernel_19724\2511944939.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['R'][idx] = np.nan
C:\Users\Marc\AppData\Local\